# 🛠️ Day 2: The Mechanics of Agentic AI
**Focus:** Tools, Binding, and the Manual Execution Loop.

---

### 🎯 Agenda
1. **Built-in Tools:** Leveraging the community ecosystem.
2. **Custom Tools & Pydantic:** Creating robust, type-safe tools.
3. **Binding:** How the LLM "sees" functions.
4. **The Message Stack:** Understanding `HumanMessage`, `AIMessage`, and `ToolMessage`.
5. **Manual Execution:** Building a ReAct loop from scratch (No `AgentExecutor`).

In [1]:
# 1. SETUP
# 1. Setup Environment
from dotenv import load_dotenv
load_dotenv()

True

!pip install pip-system-certs

In [7]:
from langchain_openai import ChatOpenAI

# Initialize the "Brain" (LLM)
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [2]:
!pip install -U ddgs

In [3]:
!pip install pip-system-certs

In [4]:

from langchain_community.tools import DuckDuckGoSearchRun

# Create the real search tool
search_tool = DuckDuckGoSearchRun()

# Test it directly (No LLM involved yet)
result = search_tool.invoke("When google antigravity launched?")
print("✅ Internet Search SUCCESS")
print(result)

DDGSException: RuntimeError: RuntimeError('error sending request for url (https://www.bing.com/search?q=When+google+antigravity+launched%3F&pq=When+google+antigravity+launched%3F&cc=wt&filters=ex1%3A%22ez5_20055_20420%22): client error (Connect)\n\nCaused by:\n    0: client error (Connect)\n    1: TLS handshake failed: cert verification failed - unable to get local issuer certificate [CERTIFICATE_VERIFY_FAILED]\n    2: [CERTIFICATE_VERIFY_FAILED]\n    3: [CERTIFICATE_VERIFY_FAILED]')

In [2]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers. Use this for math operations."""
    return a * b

@tool
def get_employee_email(name: str) -> str:
    """Get the email address of an employee by their full name."""
    # Mock Database
    db = {"Sharad Rajore": "sharad.r@zensar.com", "Alice Smith": "alice@zensar.com"}
    return db.get(name, "Email not found")

print(f"Tool Name: {multiply.name}")
print(f"Tool Description: {multiply.description}")
print(f"Tool Args Schema: {multiply.args}")

Tool Name: multiply
Tool Description: Multiply two integers. Use this for math operations.
Tool Args Schema: {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [3]:
# TAVILY SEARCH TOOL - Beginner Friendly Version
# Step 1: Install the Tavily package
#pip install tavily-python

# Step 2: Import libraries
import os
from langchain_community.tools.tavily_search import TavilySearchResults

# Step 3: Get your API key
# Go to https://tavily.com and sign up for a FREE API key
if not os.environ.get("TAVILY_API_KEY"):
    api_key = input("Paste your Tavily API Key here: ")
    os.environ["TAVILY_API_KEY"] = api_key

# Step 4: Create the search tool
tavily_tool = TavilySearchResults(max_results=3)

# Step 5: Test it with a simple search
print("Testing Tavily Search...")
result = tavily_tool.invoke("Who is the CEO of Zensar Technologies?")
print("\n✅ Search Results:")
print(result)

C:\Users\sr43993\AppData\Local\Temp\ipykernel_18196\4111713556.py:16: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tavily_tool = TavilySearchResults(max_results=3)


Testing Tavily Search...

✅ Search Results:
SSLError(MaxRetryError("HTTPSConnectionPool(host='api.tavily.com', port=443): Max retries exceeded with url: /search (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))"))


## Part 2: Custom Tools (The `@tool` Decorator)
When we need custom logic, we use the `@tool` decorator. 
**CRITICAL:** The docstring is the prompt.

In [4]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers. Use this for math operations."""
    return a * b

@tool
def get_employee_email(name: str) -> str:
    """Get the email address of an employee by their full name."""
    # Mock Database
    db = {"Sharad Rajore": "sharad.rajore@zensar.com", "Alice Smith": "alice@zensar.com"}
    return db.get(name, "Email not found")

print(f"Tool Name: {multiply.name}")
print(f"Tool Description: {multiply.description}")
print(f"Tool Args Schema: {multiply.args}")

Tool Name: multiply
Tool Description: Multiply two integers. Use this for math operations.
Tool Args Schema: {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


## Part 3: Advanced Tools (StructuredTool & Pydantic)
What if we need complex inputs? Like a tool that takes a JSON object?
We use **Pydantic** to enforce data types.

In [6]:
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool

# 1. Define the Schema (The "Contract")
class TicketInput(BaseModel):
    issue_type: str = Field(description="The type of issue: 'Hardware' or 'Software'")
    priority: int = Field(description="Priority level (1-5)")
    description: str = Field(description="Detailed description of the problem")

# 2. Define the Function
def create_it_ticket(issue_type: str, priority: int, description: str) -> str:
    return f"TICKET CREATED: [{issue_type}] Priority {priority} - {description}"

# 3. Create the Tool
ticket_tool = StructuredTool.from_function(
    func=create_it_ticket,
    name="create_ticket",
    description="Use this to log IT support tickets.",
    args_schema=TicketInput
)

# Test it manually to see Pydantic in action
print(ticket_tool.invoke({"issue_type": "Software", "priority": "1", "description": "Laptop crashed"}))

TICKET CREATED: [Software] Priority 1 - Laptop crashed


## Part 4: Binding (The Handshake)
The LLM doesn't know these functions exist yet. We must **BIND** them.
This converts Python functions -> OpenAI JSON Schema.

In [12]:
tools = [multiply, get_employee_email, ticket_tool]

# Create a new LLM instance that KNOWS about the tools
llm_with_tools = llm.bind_tools(tools)

# Let's inspect what happens when we call it
query = "My laptop screen is broken. It's urgent!"
response = llm_with_tools.invoke(query)

#query = "what is 2*2"
#response = llm_with_tools.invoke(query)

print("--- RAW LLM RESPONSE ---")
print(f"Content: {response.content}")
print(f"Tool Calls: {response.tool_calls}")

--- RAW LLM RESPONSE ---
Content: 
Tool Calls: [{'name': 'create_ticket', 'args': {'issue_type': 'Hardware', 'priority': 1, 'description': 'The laptop screen is broken and needs immediate attention.'}, 'id': 'call_39xM3IKIBRk9N4FWRDUwtLGf', 'type': 'tool_call'}]


## Part 5: The Manual Execution Loop
This is the most important part of Day 2. We will NOT use `AgentExecutor`. We will build the loop manually to understand **Message History**.

**The Cycle:**
1. **HumanMessage:** User input.
2. **AIMessage:** LLM decides to call a tool.
3. **ToolMessage:** We run the tool and pass the result back.
4. **AIMessage:** LLM interprets the result and answers.

In [21]:
from langchain_core.messages import HumanMessage, ToolMessage

# 1. User asks a question
messages = [HumanMessage(content="Who is the pm of India? And multiply 55 * 10.")]

# 2. First LLM Call
ai_msg = llm_with_tools.invoke(messages)
print(ai_msg)
messages.append(ai_msg) # IMPORTANT: Add AI's thought to history

print(f"AI Thought: {ai_msg.tool_calls}")



content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 149, 'total_tokens': 182, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_672b6a21ba', 'id': 'chatcmpl-CgupSs7eAuYThVDwh4fsr2DsP5BX1', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--db1f14c3-a099-4d08-acdb-28cdd35dbe4c-0' tool_calls=[{'name': 'multiply', 'args': {'a': 55, 'b': 10}, 'id': 'call_tPrhKmr3nu10y5xAw0qSy5BC', 'type': 'tool_call'}] usage_metadata={'input_tokens': 149, 'output_tokens': 33, 'total_tokens': 182, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
AI Thought: [{'name': 'multiply', 'args': {'a': 55, 'b': 

In [22]:
# 3. Execute Tools Manually
for tool_call in ai_msg.tool_calls:
    selected_tool = {
        "multiply": multiply, 
        "get_employee_email": get_employee_email
    }[tool_call["name"]]
    
    # Run the tool
    tool_output = selected_tool.invoke(tool_call["args"])
    
    # Create the ToolMessage
    tool_msg = ToolMessage(tool_output, tool_call_id=tool_call["id"])
    messages.append(tool_msg)
    print(f"Executed {tool_call['name']} -> Result: {tool_output}")



Executed multiply -> Result: 550


In [23]:
# 4. Second LLM Call (With full history)
final_response = llm_with_tools.invoke(messages)
print(f"\nFinal Answer: {final_response.content}")


Final Answer: The Prime Minister of India is Narendra Modi. The result of multiplying 55 by 10 is 550.
